In [2]:
# Deps
from functools import partial
import json
# Models
from mlp.genetic_algorithm import GeneticAlgorithm
from mlp.multilayer_perceptron import MultilayerPerceptron
from minimax.minimax import Minimax
# Game
from tictactoe.board import Board as tictactoe

In [7]:

def evaluate_fitness(model: MultilayerPerceptron, minimax: Minimax, chromosome: list) -> float:
    """
    Função de aptidão para o Algoritmo Genético.

    Parâmetros:
    -----------
    model : MultilayerPerceptron
        Rede MLP que será ajustada.

    minimax : Minimax
        Adversário controlado por algoritmo de decisão.

    chromosome : list
        Vetor linear de pesos da rede MLP.

    Retorna:
    --------
    float : valor da aptidão com base no resultado do jogo.
        +10 vitória do MLP
        +1 empate
        -5 derrota
        -20 jogada inválida
    """
    board, insights = play(model, minimax)
    model.load_weights_from_vector(board, chromosome)

    if 'error' in insights and insights['error'] == 'p1':
        return -20  # Jogada inválida do MLP

    match insights['result']:
        case 3: return 10  # X_WIN (MLP)
        case 1: return 1   # DRAW
        case 0: return -5  # O_WIN (Minimax)
        case _: return 0   # fallback

def play(p1, p2) -> dict:
    """
    Executa uma partida entre dois agentes com método predict().

    Retorna:
    --------
    dict com resultado e estado final do tabuleiro.
    """
    ttt = tictactoe()

    while True:
        if not ttt.update_board(1,p1.predict(ttt.board)):
            return ttt.board, {'error': 'p1', 'reason': 'P1 invalid play', 'result': -1}
        if not ttt.is_ongoing():
            break

        if not ttt.update_board(-1, p2.predict(ttt.board)):
            return ttt.board, {'error': 'p2', 'reason': 'P2 invalid play', 'result': -1}
        if not ttt.is_ongoing():
            break

    return ttt.board, {'result': ttt.check_win()}

def train(topology:list[int], population_size:int, pipeline=list[str], max_iter:int=100, threshold:float=0.95, learning_rate:float=0.1, verbose:bool=False, to_json:bool=True) -> MultilayerPerceptron:
    # Definição da arquitetura da MLP
    model = MultilayerPerceptron(topology)
    chromosome_size = model.count_weights()

    print(f'Partidas={pipeline}')
    minimax = Minimax()
    fitness_function = partial(evaluate_fitness, model, minimax)
    ag = GeneticAlgorithm(
        pop_size=population_size,
        chromosome_size=chromosome_size,
        fitness_function=fitness_function,
        max_iter=max_iter,
        learning_rate=learning_rate
    )

    for mode in pipeline:
        print(f'\n--- Treinando contra adversário no modo: {mode.upper()} ---')

        minimax.update_mode(mode)
        ag.run(verbose=verbose, threshold=threshold)

    if to_json:
        with open('model.json', '+w') as f:
            f.write(json.dumps(model.to_json()))
    return model

In [9]:
PIPELINE = (
    3 * ['medium'] +
    5 * ['hard']
)

model = train(
    topology=[9, 9 ,9],
    population_size=10,
    pipeline=PIPELINE,
    max_iter=10
    )

Partidas=['medium', 'medium', 'medium', 'hard', 'hard', 'hard', 'hard', 'hard']


TypeError: GeneticAlgorithm.__init__() got an unexpected keyword argument 'learning_rate'

In [6]:
model.predict([0, -1, -1, 0, 0, 0, 0, 0, 0])

6